In [3]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# 1. autograd

# Create tensors
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)


# Build a computational graph
y = w * x + b

# Compute gradients
y.backward()

# Print out the gradients
print(x.grad)  # x.grad = 2
print(w.grad)  # w.grad = 1
print(b.grad)  # b.grad = 1



tensor(2.)
tensor(1.)
tensor(1.)


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [13]:
# 2. autograd

# Create tensors of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer
linear = nn.Linear(3, 2)
print('w: ', linear.weight)
print('b: ', linear.bias)

# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print('dL/dW: ', linear.weight.grad)
print('dL/db ', linear.bias.grad)

# 1-step geadient descent
optimizer.step()

# You can alos perform gradient at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)

# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())


w:  Parameter containing:
tensor([[ 0.0172,  0.3065,  0.3823],
        [-0.2713,  0.3330, -0.0466]], requires_grad=True)
b:  Parameter containing:
tensor([0.4004, 0.4398], requires_grad=True)
loss:  0.8997387886047363
dL/dW:  tensor([[ 0.3844,  0.3814, -0.1334],
        [-0.3539,  0.0644, -0.0896]])
dL/db  tensor([ 0.2328, -0.1677])
loss after 1 step optimization:  0.8944565057754517


In [15]:
# 3. Loading data from numpy

# Create a numpy array
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array
z = y.numpy()

In [ ]:
# 4. Input pipeline

# Download and construct CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, transform=transforms.ToTensor(), download=True)

# Fetch one data pair (read data from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

# Data loader (this provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# When iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)

# Mini-batch images and labels
images, labels = data_iter.next()

# Actual usage of the data is as below.
for images, labels in train_loader:
    # Train code shoule be written here.
    pass

In [ ]:
# 5. Input pipeline for custom dataset

# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initalize file paths or a list of file name
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Trasform)
        # 3. Return a data pair (e.g. image and label)
        pass
  
    def __len__(self):
        # You should change 0 to the total size of your dataset
        return 0
    
# You can then use the prebuilt data loader.
custom_dataset = CustomDataset()
train_loader = torch.untils.data.DataLoader(dataset=custom_dataset, batch_size=64, shuffle=True)

In [7]:
# 6. Pretrained model

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
print(images.shape)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


/home/yitong/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/yitong/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([64, 3, 224, 224])
torch.Size([64, 100])


In [8]:
# 7. Save and load the model

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended)

torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>